In [1]:
import urllib
import requests
import pandas as pd
import json
import base64

In [2]:
text = 'david :%&= silva'
urllib.parse.quote_plus(text)

'david+%3A%25%26%3D+silva'

In [3]:
query = 'artist:silva genre:pop tag:new'
url_query = urllib.parse.quote_plus(query)

In [4]:
def get_session_token(SessionID, SessionKey):
    url = 'https://accounts.spotify.com/api/token'
    data = {'grant_type':'client_credentials'}
    encoded_key = base64.b64encode(str(SessionID + ":" + SessionKey).\
                                   encode("ascii"))
    header = {'Authorization': 'Basic {}'.format(encoded_key.decode("ascii"))}
    response = requests.post(url,
                            data = data,
                            headers = header)
    print(response.status_code)
    return response.json()['access_token']

In [6]:
keys = pd.read_csv(r"../..\Dropbox\Kent\EMAT-32210_Data-in-EMAT\Spotify-Keys_9-12-23.txt")

In [6]:
# keys

In [7]:
access_token = get_session_token(keys['Client_ID'].iloc[0], keys['Client_Secret'].iloc[0])

200


In [8]:
# access_token
api_header = {'Authorization': 'Bearer {}'.format(access_token)}

In [32]:
api_base_url = 'https://api.spotify.com/v1/'
search_url = api_base_url + 'search'
album_tracks_url = api_base_url + 'albums/{}/tracks'
track_features_url = api_base_url + 'audio-features'
tracks_url = api_base_url + 'tracks'

In [10]:
search_query1 = urllib.parse.quote_plus('artist=sabrina carpenter')
search_query2 = urllib.parse.quote_plus('artist=chappell roan')
search_params = '?q={}&type=album&market=US&limit=2'
search_params.format(search_query1)
# search_params

'?q=artist%3Dsabrina+carpenter&type=album&market=US&limit=2'

In [11]:
def api_request(endpoint_url, header):
    response = requests.get(endpoint_url, headers = header)
    print(response.status_code)
    return response

In [12]:
search_response = api_request(search_url + search_params.format(search_query1), api_header)

200


In [19]:
search_response.json()['albums']['items'][0]['id']

'3iPSVi54hsacKKl1xIR2eH'

In [14]:
cr_tracks = api_request(tracks_url.format('0EiI8ylL0FmWWpgHVTsZjZ'), api_header)
sc_tracks = api_request(tracks_url.format('5bBaoign62r1i7OV8w7mi9'), api_header)

200
200


In [17]:
cr_tracks.json()['items'][0].keys()

dict_keys(['artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_urls', 'href', 'id', 'name', 'track_number', 'type', 'uri', 'is_local'])

Popularity for the tracks and then the audio features for the tracks, then merge the dataframes on id

In [25]:
cr_tracks_df = pd.DataFrame(cr_tracks.json()['items'])
cr_tracks_df.head()

,artists,available_markets,disc_number,duration_ms,explicit,external_urls,href,id,name,track_number,type,uri,is_local
0,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,219710,True,{'spotify': 'https://open.spotify.com/track/53...,https://api.spotify.com/v1/tracks/53IRnAWx13PY...,53IRnAWx13PYmoVYtemUBS,Femininomenon,1,track,spotify:track:53IRnAWx13PYmoVYtemUBS,False
1,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,192720,True,{'spotify': 'https://open.spotify.com/track/7F...,https://api.spotify.com/v1/tracks/7FOgcfdz9Nx5...,7FOgcfdz9Nx5V9lCNXdBYv,Red Wine Supernova,2,track,spotify:track:7FOgcfdz9Nx5V9lCNXdBYv,False
2,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,204857,False,{'spotify': 'https://open.spotify.com/track/4r...,https://api.spotify.com/v1/tracks/4rlQza35DE4P...,4rlQza35DE4Prh5yonxnCs,After Midnight,3,track,spotify:track:4rlQza35DE4Prh5yonxnCs,False
3,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,205520,False,{'spotify': 'https://open.spotify.com/track/6G...,https://api.spotify.com/v1/tracks/6GdUMm5xMw9h...,6GdUMm5xMw9hJckIc0qsGb,Coffee,4,track,spotify:track:6GdUMm5xMw9hJckIc0qsGb,False
4,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,232796,True,{'spotify': 'https://open.spotify.com/track/3W...,https://api.spotify.com/v1/tracks/3WSOUb3U7tqU...,3WSOUb3U7tqURbBSgZTrZX,Casual,5,track,spotify:track:3WSOUb3U7tqURbBSgZTrZX,False


In [29]:
','.join(cr_tracks_df['id'].to_list())

'53IRnAWx13PYmoVYtemUBS,7FOgcfdz9Nx5V9lCNXdBYv,4rlQza35DE4Prh5yonxnCs,6GdUMm5xMw9hJckIc0qsGb,3WSOUb3U7tqURbBSgZTrZX,1rNSCrsOoWyhKH4g47mehU,4xdBrk0nFZaP54vvZj0yx7,32fSZSbxeVoiZShMQKLc6Z,5KtvumPgVZmt8wg9xONE0T,4ROYC4vHfPZ28mqz0eLrzL,1k2pQc5i348DCHwbn5KTdc,4LKYOetuIF5c9XjeLBL9av,6dA1JQt3f3meGzahhNN75m,0q8TNCQhwnDwnzq38IVV4l'

In [30]:
tracks_params_cr = '?market=US&ids={}'.format(','.join(cr_tracks_df['id'].to_list()))

In [37]:
tracks_url + tracks_params_cr

'https://api.spotify.com/v1/tracks?market=US&ids=53IRnAWx13PYmoVYtemUBS,7FOgcfdz9Nx5V9lCNXdBYv,4rlQza35DE4Prh5yonxnCs,6GdUMm5xMw9hJckIc0qsGb,3WSOUb3U7tqURbBSgZTrZX,1rNSCrsOoWyhKH4g47mehU,4xdBrk0nFZaP54vvZj0yx7,32fSZSbxeVoiZShMQKLc6Z,5KtvumPgVZmt8wg9xONE0T,4ROYC4vHfPZ28mqz0eLrzL,1k2pQc5i348DCHwbn5KTdc,4LKYOetuIF5c9XjeLBL9av,6dA1JQt3f3meGzahhNN75m,0q8TNCQhwnDwnzq38IVV4l'

In [42]:
cr_track_response = api_request(tracks_url + tracks_params_cr, api_header)
cr_track_features_response = api_request(track_features_url + '?ids={}'.format(','.join(cr_tracks_df['id'].to_list())), api_header)

200
200


In [43]:
cr_track_response.json().keys()

dict_keys(['tracks'])

In [47]:
pd.DataFrame(cr_track_response.json()['tracks']).head()

,album,artists,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,is_playable,name,popularity,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,219710,True,{'isrc': 'USUG12302738'},{'spotify': 'https://open.spotify.com/track/53...,https://api.spotify.com/v1/tracks/53IRnAWx13PY...,53IRnAWx13PYmoVYtemUBS,False,True,Femininomenon,85,1,track,spotify:track:53IRnAWx13PYmoVYtemUBS
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,192720,True,{'isrc': 'USUG12301845'},{'spotify': 'https://open.spotify.com/track/7F...,https://api.spotify.com/v1/tracks/7FOgcfdz9Nx5...,7FOgcfdz9Nx5V9lCNXdBYv,False,True,Red Wine Supernova,88,2,track,spotify:track:7FOgcfdz9Nx5V9lCNXdBYv
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,204857,False,{'isrc': 'USUG12302739'},{'spotify': 'https://open.spotify.com/track/4r...,https://api.spotify.com/v1/tracks/4rlQza35DE4P...,4rlQza35DE4Prh5yonxnCs,False,True,After Midnight,80,3,track,spotify:track:4rlQza35DE4Prh5yonxnCs
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,205520,False,{'isrc': 'USUG12305909'},{'spotify': 'https://open.spotify.com/track/6G...,https://api.spotify.com/v1/tracks/6GdUMm5xMw9h...,6GdUMm5xMw9hJckIc0qsGb,False,True,Coffee,74,4,track,spotify:track:6GdUMm5xMw9hJckIc0qsGb
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,232796,True,{'isrc': 'QM24S2205365'},{'spotify': 'https://open.spotify.com/track/3W...,https://api.spotify.com/v1/tracks/3WSOUb3U7tqU...,3WSOUb3U7tqURbBSgZTrZX,False,True,Casual,87,5,track,spotify:track:3WSOUb3U7tqURbBSgZTrZX


In [51]:
cr_track_features_response.json()['audio_features'][0]

{'danceability': 0.609,
 'energy': 0.758,
 'key': 0,
 'loudness': -4.885,
 'mode': 0,
 'speechiness': 0.0525,
 'acousticness': 0.139,
 'instrumentalness': 5.38e-06,
 'liveness': 0.11,
 'valence': 0.236,
 'tempo': 124.949,
 'type': 'audio_features',
 'id': '53IRnAWx13PYmoVYtemUBS',
 'uri': 'spotify:track:53IRnAWx13PYmoVYtemUBS',
 'track_href': 'https://api.spotify.com/v1/tracks/53IRnAWx13PYmoVYtemUBS',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/53IRnAWx13PYmoVYtemUBS',
 'duration_ms': 219711,
 'time_signature': 4}

In [52]:
pd.DataFrame(cr_track_features_response.json()['audio_features']).head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.609,0.758,0,-4.885,0,0.0525,0.1390,0.000005,0.1100,0.236,124.949,audio_features,53IRnAWx13PYmoVYtemUBS,spotify:track:53IRnAWx13PYmoVYtemUBS,https://api.spotify.com/v1/tracks/53IRnAWx13PY...,https://api.spotify.com/v1/audio-analysis/53IR...,219711,4
1,0.657,0.820,9,-4.448,1,0.0441,0.0176,0.000000,0.0847,0.709,124.041,audio_features,7FOgcfdz9Nx5V9lCNXdBYv,spotify:track:7FOgcfdz9Nx5V9lCNXdBYv,https://api.spotify.com/v1/tracks/7FOgcfdz9Nx5...,https://api.spotify.com/v1/audio-analysis/7FOg...,192721,4
2,0.708,0.891,0,-3.826,1,0.0777,0.0248,0.000000,0.0578,0.840,110.058,audio_features,4rlQza35DE4Prh5yonxnCs,spotify:track:4rlQza35DE4Prh5yonxnCs,https://api.spotify.com/v1/tracks/4rlQza35DE4P...,https://api.spotify.com/v1/audio-analysis/4rlQ...,204857,4
3,0.434,0.219,2,-10.200,1,0.0301,0.9480,0.000000,0.0851,0.234,82.602,audio_features,6GdUMm5xMw9hJckIc0qsGb,spotify:track:6GdUMm5xMw9hJckIc0qsGb,https://api.spotify.com/v1/tracks/6GdUMm5xMw9h...,https://api.spotify.com/v1/audio-analysis/6GdU...,205520,4
4,0.508,0.411,1,-6.950,1,0.0269,0.2260,0.000000,0.0571,0.340,131.008,audio_features,3WSOUb3U7tqURbBSgZTrZX,spotify:track:3WSOUb3U7tqURbBSgZTrZX,https://api.spotify.com/v1/tracks/3WSOUb3U7tqU...,https://api.spotify.com/v1/audio-analysis/3WSO...,232796,4
